In [1]:
import multiprocess as mp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math
from tqdm import tqdm
import requests
from urllib.parse import urlparse
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import datetime
from sklearn.preprocessing import quantile_transform
pd.options.mode.chained_assignment = None  # default='warn'
import gc

In [2]:
def bonbubun_allocator(x):
    # x is a string
    try:
        if pd.isna(x) == True:
            return x
        else:
            return str(int(x))
    except:
        return np.nan

In [3]:
def buildinggroup_modifier(x):
    # x is a string
    
    try:
        return int(x)
    except:
        return x

In [4]:
def add_0(x):
    if pd.isnull(x) == False:
        splitted = x.split('-')
        if len(splitted) == 1:
            return x+'-0'
        else:
            return x
    else:
        return x

In [9]:
def read_coor_data4():
    basedir = './좌표데이터4/'
    filenames = [f for f in os.listdir(basedir) if (f.endswith('.txt'))&(f.startswith('entrc_seoul'))]
    
    colnames = ['시군구코드', '출입구일련번호', '법정동코드', '시도명', '시군구명', '읍면동명', '도로명코드', '도로명',
                '지하여부', '건물본번', '건물부번', '건물명', '우편번호', '건물용도분류', '건물군여부', '관할행정동',
                'X좌표', 'Y좌표'
               ]
    
    dfs_list = []
    for file in tqdm(filenames):
        try:
            df = pd.read_csv(basedir + file, sep='|', header=None, names=colnames)
        except:
            try:
                df = pd.read_csv(basedir + file, sep='|', header=None, names=colnames, encoding='euc-kr')
            except:
                try:
                    df = pd.read_csv(basedir + file, sep='|', header=None, names=colnames, encoding='ansi')
                except:
                    raise ValueError('myerror')
                    
                    
        dfs_list.append(df)
        
    concat_df = pd.concat(dfs_list).reset_index(drop=True)
    
    return concat_df

In [10]:
prac_coor_df = read_coor_data4()
print(prac_coor_df.shape)
prac_coor_df.head()

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.25it/s]

(546776, 18)


,시군구코드,출입구일련번호,법정동코드,시도명,시군구명,읍면동명,도로명코드,도로명,지하여부,건물본번,건물부번,건물명,우편번호,건물용도분류,건물군여부,관할행정동,X좌표,Y좌표
0,11110,760,1111010100,서울특별시,종로구,청운동,111103100012,자하문로,0,94,0,NaN,3047,근린생활시설,0,청운효자동,953241.683263,1.954023e+06
1,11110,4933,1111010100,서울특별시,종로구,청운동,111103100012,자하문로,0,96,0,평안빌,3047,주택,0,청운효자동,953243.258151,1.954034e+06
2,11110,287,1111010100,서울특별시,종로구,청운동,111103100012,자하문로,0,98,0,청운빌라,3047,주택,0,청운효자동,953240.877646,1.954055e+06
3,11110,1336,1111010100,서울특별시,종로구,청운동,111103100012,자하문로,0,99,3,풍림팍사이드빌라,3032,주택,1,청운효자동,953195.459364,1.954079e+06
4,11110,6021,1111010100,서울특별시,종로구,청운동,111103100012,자하문로,0,99,4,경복빌라,3032,주택,1,청운효자동,953193.346779,1.954097e+06


In [11]:
prac_coor_df[['건물본번', '건물부번']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 546776 entries, 0 to 546775
Data columns (total 2 columns):
건물본번    546776 non-null int64
건물부번    546776 non-null int64
dtypes: int64(2)
memory usage: 8.3 MB


In [12]:
main_data_df = pd.read_csv('./prepped_data/main_data_baseline_checkpoint_1.csv')
print(main_data_df.shape)

(4553162, 18)


In [13]:
main_data_df['도로명주소'].nunique()

112020

In [16]:
def roadname_addr_modifier(x):
    # x is a string
    
    if pd.isna(x) == True:
        return x
    else:
        splitted = x.split('-')
        if len(splitted) == 1:
            return x + '-0'
        else:
            return x

In [17]:
main_data_df['도로명주소'] = main_data_df['도로명주소'].apply(roadname_addr_modifier)
main_data_df['도로명주소'].unique()[:50]

array(['서울특별시 강남구 언주로 103-0', '서울특별시 강남구 언주로 3-0', '서울특별시 강남구 개포로 307-0',
       '서울특별시 강남구 개포로109길 69-0', '서울특별시 강남구 개포로 310-0',
       '서울특별시 강남구 선릉로 7-0', '서울특별시 강남구 삼성로 11-0', '서울특별시 강남구 삼성로 14-0',
       '서울특별시 강남구 삼성로4길 17-0', '서울특별시 강남구 개포로 516-0',
       '서울특별시 강남구 언주로 105-0', '서울특별시 강남구 언주로 110-0',
       '서울특별시 강남구 논현로2길 62-0', '서울특별시 강남구 개포로109길 21-0',
       '서울특별시 강남구 논현로2길 36-0', '서울특별시 강남구 개포로109길 9-0',
       '서울특별시 강남구 언주로7길 23-0', '서울특별시 강남구 개포로 411-0',
       '서울특별시 강남구 개포로 311-0', '서울특별시 강남구 개포로31길 3-12',
       '서울특별시 강남구 개포로31길 9-5', '서울특별시 강남구 선릉로 8-0', '서울특별시 강남구 개포로 303-0',
       '서울특별시 강남구 언주로 107-0', '서울특별시 강남구 개포로 409-0',
       '서울특별시 강남구 개포로28길 28-0', '서울특별시 강남구 논현로2길 38-0',
       '서울특별시 강남구 논현로2길 50-0', '서울특별시 강남구 논현로2길 22-0',
       '서울특별시 강남구 양재대로 379-0', '서울특별시 강남구 개포로15길 32-8',
       '서울특별시 강남구 개포로 405-0', '서울특별시 강남구 논현로2길 34-0',
       '서울특별시 강남구 언주로 101-0', '서울특별시 강남구 선릉로8길 5-0',
       '서울특별시 강남구 선릉로18길 12-0', '서울특별시 강남구 논현로2길 61-4',
       '서울특

In [18]:
prac_coor_df['지번주소'] = prac_coor_df['시도명'] + ' ' + prac_coor_df['시군구명'] + ' ' + prac_coor_df['도로명'] + ' '\
+ prac_coor_df['건물본번'].apply(str) + '-' + prac_coor_df['건물부번'].apply(str)

In [19]:
prac_coor_df['지번주소'].unique()[:50]

array(['서울특별시 종로구 자하문로 94-0', '서울특별시 종로구 자하문로 96-0',
       '서울특별시 종로구 자하문로 98-0', '서울특별시 종로구 자하문로 99-3',
       '서울특별시 종로구 자하문로 99-4', '서울특별시 종로구 자하문로 100-0',
       '서울특별시 종로구 자하문로 100-1', '서울특별시 종로구 자하문로 101-0',
       '서울특별시 종로구 자하문로 103-0', '서울특별시 종로구 자하문로 104-0',
       '서울특별시 종로구 자하문로 105-0', '서울특별시 종로구 자하문로 106-0',
       '서울특별시 종로구 자하문로 106-3', '서울특별시 종로구 자하문로 106-4',
       '서울특별시 종로구 자하문로 108-0', '서울특별시 종로구 자하문로 108-1',
       '서울특별시 종로구 자하문로 110-0', '서울특별시 종로구 자하문로 112-0',
       '서울특별시 종로구 자하문로 114-0', '서울특별시 종로구 자하문로 115-0',
       '서울특별시 종로구 자하문로 115-14', '서울특별시 종로구 자하문로 115-16',
       '서울특별시 종로구 자하문로 115-18', '서울특별시 종로구 자하문로 115-22',
       '서울특별시 종로구 자하문로 116-0', '서울특별시 종로구 자하문로 116-1',
       '서울특별시 종로구 자하문로 118-0', '서울특별시 종로구 자하문로 118-1',
       '서울특별시 종로구 자하문로 119-0', '서울특별시 종로구 자하문로 119-1',
       '서울특별시 종로구 자하문로 120-0', '서울특별시 종로구 자하문로 121-0',
       '서울특별시 종로구 자하문로 121-1', '서울특별시 종로구 자하문로 122-0',
       '서울특별시 종로구 자하문로 124-0', '서울특별시 종로구 자하문로 125-0',
       '서울특

In [20]:
prac_coor_df['지번주소'].nunique()

546730

In [22]:
prac_coor_df.shape

(546776, 19)

In [21]:
coor_df = prac_coor_df[['지번주소', 'X좌표', 'Y좌표']].drop_duplicates()
coor_df.shape

(546776, 3)

In [24]:
coor_df.rename(columns={'지번주소':'도로명주소'}, inplace=True)

In [25]:
coor_df.head()

,도로명주소,X좌표,Y좌표
0,서울특별시 종로구 자하문로 94-0,953241.683263,1.954023e+06
1,서울특별시 종로구 자하문로 96-0,953243.258151,1.954034e+06
2,서울특별시 종로구 자하문로 98-0,953240.877646,1.954055e+06
3,서울특별시 종로구 자하문로 99-3,953195.459364,1.954079e+06
4,서울특별시 종로구 자하문로 99-4,953193.346779,1.954097e+06


In [27]:
merge_df = main_data_df.merge(coor_df, on=['도로명주소'])
merge_df.shape

(4441256, 20)

In [28]:
merge_df.head()

,Unnamed: 0,전월세매매구분,전용면적(㎡),계약일,금액(만원),월세(만원),층,건축년도,가격/면적,년,월,target_log_transformed,target/area_log_transformed,건물종류,지번주소,대장구분명,도로명주소,addr_split_len,X좌표,Y좌표
0,0,매매,77.75,8,57000,0,2,1988,733.118971,2013,9,10.950807,6.597308,아파트,서울특별시 강남구 개포동 655-2,일반,서울특별시 강남구 언주로 103-0,4.0,960473.743425,1.942553e+06
1,1,매매,77.75,16,57000,0,2,1988,733.118971,2013,12,10.950807,6.597308,아파트,서울특별시 강남구 개포동 655-2,일반,서울특별시 강남구 언주로 103-0,4.0,960473.743425,1.942553e+06
2,862,매매,77.75,29,55000,0,7,1988,707.395498,2014,1,10.915088,6.561590,아파트,서울특별시 강남구 개포동 655-2,일반,서울특별시 강남구 언주로 103-0,4.0,960473.743425,1.942553e+06
3,863,매매,77.75,10,59500,0,4,1988,765.273312,2014,11,10.993732,6.640233,아파트,서울특별시 강남구 개포동 655-2,일반,서울특별시 강남구 언주로 103-0,4.0,960473.743425,1.942553e+06
4,1785,매매,77.75,28,75000,0,5,1988,964.630225,2016,3,11.225243,6.871745,아파트,서울특별시 강남구 개포동 655-2,일반,서울특별시 강남구 언주로 103-0,4.0,960473.743425,1.942553e+06


In [29]:
merge_df.drop(columns=['addr_split_len', '대장구분명'], inplace=True)
merge_df.head()

,Unnamed: 0,전월세매매구분,전용면적(㎡),계약일,금액(만원),월세(만원),층,건축년도,가격/면적,년,월,target_log_transformed,target/area_log_transformed,건물종류,지번주소,도로명주소,X좌표,Y좌표
0,0,매매,77.75,8,57000,0,2,1988,733.118971,2013,9,10.950807,6.597308,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,960473.743425,1.942553e+06
1,1,매매,77.75,16,57000,0,2,1988,733.118971,2013,12,10.950807,6.597308,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,960473.743425,1.942553e+06
2,862,매매,77.75,29,55000,0,7,1988,707.395498,2014,1,10.915088,6.561590,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,960473.743425,1.942553e+06
3,863,매매,77.75,10,59500,0,4,1988,765.273312,2014,11,10.993732,6.640233,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,960473.743425,1.942553e+06
4,1785,매매,77.75,28,75000,0,5,1988,964.630225,2016,3,11.225243,6.871745,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,960473.743425,1.942553e+06


In [30]:
merge_df.drop(columns=['Unnamed: 0'], inplace=True)

In [31]:
merge_df.head()

,전월세매매구분,전용면적(㎡),계약일,금액(만원),월세(만원),층,건축년도,가격/면적,년,월,target_log_transformed,target/area_log_transformed,건물종류,지번주소,도로명주소,X좌표,Y좌표
0,매매,77.75,8,57000,0,2,1988,733.118971,2013,9,10.950807,6.597308,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,960473.743425,1.942553e+06
1,매매,77.75,16,57000,0,2,1988,733.118971,2013,12,10.950807,6.597308,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,960473.743425,1.942553e+06
2,매매,77.75,29,55000,0,7,1988,707.395498,2014,1,10.915088,6.561590,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,960473.743425,1.942553e+06
3,매매,77.75,10,59500,0,4,1988,765.273312,2014,11,10.993732,6.640233,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,960473.743425,1.942553e+06
4,매매,77.75,28,75000,0,5,1988,964.630225,2016,3,11.225243,6.871745,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,960473.743425,1.942553e+06


In [32]:
merge_df.to_csv('./prepped_data/main_data_baseline_checkpoint_2.csv', index=False)